# Packaging Projects for Distribution

- Creating a basic Python project with `setup.py` and `setup.cfg`
- Specifying dependencies
- Activating projects in a virtualenv with `setup.py develop`
- Distributing data with your project
- Using entry points to create console scripts
- Uploading source distributions to PyPI

# First, some terminology...

- a Python **module** is typically a single file ending in `.py` located somewhere along `sys.path` that you can use with the Python `import` statement
- a Python **package** is a folder located somewhere along `sys.path` containing a "magic" file `__init__.py` which can also be imported. If you import a package, Python is actually importing the `__init__.py` *module* in that *package*. You can also import modules or subpackages from a package.
- a Python **project** is a unit of distribution of Python code (it's something you can `pip install`)

# Creating a basic Python project with `setup.py` and `setup.cfg`

To create a project for distribution, you'll need to create a directory with:

- one or more Python packages to distribute
- a `setup.py` file
- (optionally) a `setup.cfg` file

In [ ]:
%%bash
rm -r data/MyProject
mkdir -p data/MyProject/mypackage

In [ ]:
%%file data/MyProject/mypackage/__init__.py
print('This is the __init__ file for mypackage')

In [ ]:
%%file data/MyProject/mypackage/mymodule.py
print('This is mymodule')


def greet(name):
    print(f'Hello, {name}!')

In [ ]:
!find data/MyProject

For this demo, we'll use `setup.cfg` to provide metadata for our project, so we only need a minimal setup.py:

In [ ]:
%%file data/MyProject/setup.py
from setuptools import setup


setup()

We can create the `setup.cfg` file to specify how `setuptools` will build and distribute our project:

In [1]:
%%file data/MyProject/setup.cfg
[metadata]
name = MyProject
version = 0.1
url = file:///
author = Some Person
author_email = somebody@example.com
description = This should be a short description of our project
long_description = file: README.md
# classifiers =
#     Programming Language :: Python :: 3
#     Programming Language :: Python :: 3.7
# keywords = test, class

Writing data/MyProject/setup.cfg


It's always nice to provide a README as well:

In [2]:
%%file data/MyProject/README.md
# MyProject

This project is a test setuptools project.

Writing data/MyProject/README.md


## Creating a source distribution

The entry point for all our project management commands is `setup.py`.

We can create a simple source distribution of our project by calling `python setup.py sdist`:

In [3]:
%%bash
cd data/MyProject
rm -r dist
python setup.py sdist

running sdist
running egg_info
creating MyProject.egg-info
writing MyProject.egg-info/PKG-INFO
writing dependency_links to MyProject.egg-info/dependency_links.txt
writing top-level names to MyProject.egg-info/top_level.txt
writing manifest file 'MyProject.egg-info/SOURCES.txt'
reading manifest file 'MyProject.egg-info/SOURCES.txt'
writing manifest file 'MyProject.egg-info/SOURCES.txt'
running check
creating MyProject-0.1
creating MyProject-0.1/MyProject.egg-info
copying files to MyProject-0.1...
copying README.md -> MyProject-0.1
copying setup.cfg -> MyProject-0.1
copying setup.py -> MyProject-0.1
copying MyProject.egg-info/PKG-INFO -> MyProject-0.1/MyProject.egg-info
copying MyProject.egg-info/SOURCES.txt -> MyProject-0.1/MyProject.egg-info
copying MyProject.egg-info/dependency_links.txt -> MyProject-0.1/MyProject.egg-info
copying MyProject.egg-info/top_level.txt -> MyProject-0.1/MyProject.egg-info
Writing MyProject-0.1/setup.cfg
creating dist
Creating tar archive
removing 'MyProject-

rm: dist: No such file or directory


In [4]:
!tar tzf data/MyProject/dist/MyProject-0.1.tar.gz

MyProject-0.1/
MyProject-0.1/MyProject.egg-info/
MyProject-0.1/MyProject.egg-info/PKG-INFO
MyProject-0.1/MyProject.egg-info/SOURCES.txt
MyProject-0.1/MyProject.egg-info/dependency_links.txt
MyProject-0.1/MyProject.egg-info/top_level.txt
MyProject-0.1/PKG-INFO
MyProject-0.1/README.md
MyProject-0.1/setup.cfg
MyProject-0.1/setup.py


## Adding our packages

So we have an empty project (no packages/modules). We need to tell setuptools to actually include our package explicitly:

In [5]:
%%file data/MyProject/setup.cfg
[metadata]
name = MyProject
url = file:///
author = Some Person
author_email = somebody@example.com
version = 0.1
description = This should be a short description of our project
long_description = file: README.md
classifiers =
    Programming Language :: Python :: 3
    Programming Language :: Python :: 3.7
keywords = test, class

[options]
packages = mypackage

Overwriting data/MyProject/setup.cfg


In [6]:
%%bash
cd data/MyProject
python setup.py sdist

running sdist
running egg_info
writing MyProject.egg-info/PKG-INFO
writing dependency_links to MyProject.egg-info/dependency_links.txt
writing top-level names to MyProject.egg-info/top_level.txt
reading manifest file 'MyProject.egg-info/SOURCES.txt'
writing manifest file 'MyProject.egg-info/SOURCES.txt'
running check
creating MyProject-0.1
creating MyProject-0.1/MyProject.egg-info
creating MyProject-0.1/mypackage
copying files to MyProject-0.1...
copying README.md -> MyProject-0.1
copying setup.cfg -> MyProject-0.1
copying setup.py -> MyProject-0.1
copying MyProject.egg-info/PKG-INFO -> MyProject-0.1/MyProject.egg-info
copying MyProject.egg-info/SOURCES.txt -> MyProject-0.1/MyProject.egg-info
copying MyProject.egg-info/dependency_links.txt -> MyProject-0.1/MyProject.egg-info
copying MyProject.egg-info/top_level.txt -> MyProject-0.1/MyProject.egg-info
copying mypackage/__init__.py -> MyProject-0.1/mypackage
copying mypackage/mymodule.py -> MyProject-0.1/mypackage
Writing MyProject-0.1/s

In [7]:
!tar tzf data/MyProject/dist/MyProject-0.1.tar.gz

MyProject-0.1/
MyProject-0.1/MyProject.egg-info/
MyProject-0.1/MyProject.egg-info/PKG-INFO
MyProject-0.1/MyProject.egg-info/SOURCES.txt
MyProject-0.1/MyProject.egg-info/dependency_links.txt
MyProject-0.1/MyProject.egg-info/top_level.txt
MyProject-0.1/PKG-INFO
MyProject-0.1/README.md
MyProject-0.1/mypackage/
MyProject-0.1/mypackage/__init__.py
MyProject-0.1/mypackage/mymodule.py
MyProject-0.1/setup.cfg
MyProject-0.1/setup.py


## Specifying dependencies

We can tell setuptools that we depend on particular versions (or version ranges) of other packages with an `install_requires` option:

In [12]:
%%file data/MyProject/setup.cfg
[metadata]
name = MyProject
url = file:///
author = Some Person
author_email = somebody@example.com
version = 0.1
description = This should be a short description of our project
long_description = file: README.md
classifiers =
    Programming Language :: Python :: 3
    Programming Language :: Python :: 3.7
keywords = test, class

[options]
packages = mypackage
install_requires = 
    numpy>=1.16.0,<1.17

Overwriting data/MyProject/setup.cfg


In [13]:
%%bash
cd data/MyProject
python setup.py sdist

running sdist
running egg_info
writing MyProject.egg-info/PKG-INFO
writing dependency_links to MyProject.egg-info/dependency_links.txt
writing requirements to MyProject.egg-info/requires.txt
writing top-level names to MyProject.egg-info/top_level.txt
reading manifest file 'MyProject.egg-info/SOURCES.txt'
writing manifest file 'MyProject.egg-info/SOURCES.txt'
running check
creating MyProject-0.1
creating MyProject-0.1/MyProject.egg-info
creating MyProject-0.1/mypackage
copying files to MyProject-0.1...
copying README.md -> MyProject-0.1
copying setup.cfg -> MyProject-0.1
copying setup.py -> MyProject-0.1
copying MyProject.egg-info/PKG-INFO -> MyProject-0.1/MyProject.egg-info
copying MyProject.egg-info/SOURCES.txt -> MyProject-0.1/MyProject.egg-info
copying MyProject.egg-info/dependency_links.txt -> MyProject-0.1/MyProject.egg-info
copying MyProject.egg-info/requires.txt -> MyProject-0.1/MyProject.egg-info
copying MyProject.egg-info/top_level.txt -> MyProject-0.1/MyProject.egg-info
copyi

In [14]:
cat data/MyProject/MyProject.egg-info/requires.txt

numpy<1.17,>=1.16.0


# Activating projects using `setup.py develop`

When we're developing our project, we probably want its packages to be importable as though it were 'installed' in our virtualenv. To do this, we can invoke `setup.py` with the `develop` option. 

This creates a `MyProject.egg-link` file in a location along `sys.path` which makes your packages importable from anwhere that uses the virtualenv.

Note:

`pip install -e .` has equivalent effect to `python setup.py develop`

In [15]:
%%bash
cd data/MyProject
rm -fr env
python -m venv env
source env/bin/activate
python setup.py develop

running develop
running egg_info
writing MyProject.egg-info/PKG-INFO
writing dependency_links to MyProject.egg-info/dependency_links.txt
writing requirements to MyProject.egg-info/requires.txt
writing top-level names to MyProject.egg-info/top_level.txt
reading manifest file 'MyProject.egg-info/SOURCES.txt'
writing manifest file 'MyProject.egg-info/SOURCES.txt'
running build_ext
Creating /Users/rick446/src/arborian-classes/data/MyProject/env/lib/python3.7/site-packages/MyProject.egg-link (link to .)
Adding MyProject 0.1 to easy-install.pth file

Installed /Users/rick446/src/arborian-classes/data/MyProject
Processing dependencies for MyProject==0.1
Searching for numpy<1.17,>=1.16.0
Reading https://pypi.org/simple/numpy/
Best match: numpy 1.16.6
Processing numpy-1.16.6-cp37-cp37m-macosx_10_9_x86_64.whl
Installing numpy-1.16.6-cp37-cp37m-macosx_10_9_x86_64.whl to /Users/rick446/src/arborian-classes/data/MyProject/env/lib/python3.7/site-packages
Adding numpy 1.16.6 to easy-install.pth file


In [16]:
cat data/MyProject/env/lib/python3.7/site-packages/easy-install.pth

/Users/rick446/src/arborian-classes/data/MyProject
./numpy-1.16.6-py3.7-macosx-10.14-x86_64.egg


In [17]:
%%bash
source data/MyProject/env/bin/activate
cd /usr/bin
python -c 'import mypackage.mymodule; mypackage.mymodule.greet("class")'

This is the __init__ file for mypackage
This is mymodule
Hello, class!


## Distributing data with our project

Normally, only Python files are included with our project. In order to include non-Python files, we need to specify those as well:

In [18]:
%%file data/MyProject/mypackage/template.txt
This is an awesome template that greets you.

Hello, ${name}!

Writing data/MyProject/mypackage/template.txt


In [19]:
%%file data/MyProject/mypackage/mymodule.py
import os, string


def greet(name):
    with open(os.path.join(
        os.path.dirname(__file__),
        'template.txt'
    )) as f:
        template = string.Template(f.read())
    print(template.safe_substitute({'name': name}))

Overwriting data/MyProject/mypackage/mymodule.py


In [20]:
%%bash
cd data/MyProject
python setup.py sdist

running sdist
running egg_info
writing MyProject.egg-info/PKG-INFO
writing dependency_links to MyProject.egg-info/dependency_links.txt
writing requirements to MyProject.egg-info/requires.txt
writing top-level names to MyProject.egg-info/top_level.txt
reading manifest file 'MyProject.egg-info/SOURCES.txt'
writing manifest file 'MyProject.egg-info/SOURCES.txt'
running check
creating MyProject-0.1
creating MyProject-0.1/MyProject.egg-info
creating MyProject-0.1/mypackage
copying files to MyProject-0.1...
copying README.md -> MyProject-0.1
copying setup.cfg -> MyProject-0.1
copying setup.py -> MyProject-0.1
copying MyProject.egg-info/PKG-INFO -> MyProject-0.1/MyProject.egg-info
copying MyProject.egg-info/SOURCES.txt -> MyProject-0.1/MyProject.egg-info
copying MyProject.egg-info/dependency_links.txt -> MyProject-0.1/MyProject.egg-info
copying MyProject.egg-info/requires.txt -> MyProject-0.1/MyProject.egg-info
copying MyProject.egg-info/top_level.txt -> MyProject-0.1/MyProject.egg-info
copyi

In [21]:
%%file data/MyProject/setup.cfg
[metadata]
name = MyProject
url = file:///
author = Some Person
author_email = somebody@example.com
version = 0.1
description = This should be a short description of our project
long_description = file: README.md
classifiers =
    Programming Language :: Python :: 3
    Programming Language :: Python :: 3.7
keywords = test, class

[options]
packages = mypackage
install_requires = 
    numpy>=1.16.0<1.17
    
[options.package_data]
* = *.txt

Overwriting data/MyProject/setup.cfg


In [22]:
%%bash
cd data/MyProject
python setup.py sdist

running sdist
running egg_info
writing MyProject.egg-info/PKG-INFO
writing dependency_links to MyProject.egg-info/dependency_links.txt
writing requirements to MyProject.egg-info/requires.txt
writing top-level names to MyProject.egg-info/top_level.txt
reading manifest file 'MyProject.egg-info/SOURCES.txt'
writing manifest file 'MyProject.egg-info/SOURCES.txt'
running check
creating MyProject-0.1
creating MyProject-0.1/MyProject.egg-info
creating MyProject-0.1/mypackage
copying files to MyProject-0.1...
copying README.md -> MyProject-0.1
copying setup.cfg -> MyProject-0.1
copying setup.py -> MyProject-0.1
copying MyProject.egg-info/PKG-INFO -> MyProject-0.1/MyProject.egg-info
copying MyProject.egg-info/SOURCES.txt -> MyProject-0.1/MyProject.egg-info
copying MyProject.egg-info/dependency_links.txt -> MyProject-0.1/MyProject.egg-info
copying MyProject.egg-info/requires.txt -> MyProject-0.1/MyProject.egg-info
copying MyProject.egg-info/top_level.txt -> MyProject-0.1/MyProject.egg-info
copyi

In [23]:
%%bash
source data/MyProject/env/bin/activate
cd /
python -c 'import mypackage.mymodule; mypackage.mymodule.greet("class")'

This is the __init__ file for mypackage
This is an awesome template that greets you.

Hello, class!



# Using entry_points for console_scripts

If you need to create a new command-line tool, a nice approach is to use the `entry_points` feature of `setuptools`:

In [24]:
%%file data/MyProject/setup.cfg
[metadata]
name = MyProject
url = file:///
author = Some Person
author_email = somebody@example.com
version = 0.1
description = This should be a short description of our project
long_description = file: README.md
classifiers =
    Programming Language :: Python :: 3
    Programming Language :: Python :: 3.7
keywords = test, class

[options]
packages = mypackage
install_requires = 
    numpy>=1.16.0<1.17
    
[options.package_data]
* = *.txt

[options.entry_points]
console_scripts =
  my-greet=mypackage.mymodule:greet_main

Overwriting data/MyProject/setup.cfg


In [25]:
%%file data/MyProject/mypackage/mymodule.py
import os, sys, string


def greet(name):
    with open(os.path.join(
        os.path.dirname(__file__),
        'template.txt'
    )) as f:
        template = string.Template(f.read())
    print(template.safe_substitute({'name': name}))
    
    
def greet_main():
    if len(sys.argv) > 1:
        name = sys.argv[1]
    else:
        name = 'unknown human'
    greet(name)

Overwriting data/MyProject/mypackage/mymodule.py


In [26]:
%%bash
cd data/MyProject
source env/bin/activate
python setup.py develop  # or pip install -e .

running develop
running egg_info
writing MyProject.egg-info/PKG-INFO
writing dependency_links to MyProject.egg-info/dependency_links.txt
writing entry points to MyProject.egg-info/entry_points.txt
writing requirements to MyProject.egg-info/requires.txt
writing top-level names to MyProject.egg-info/top_level.txt
reading manifest file 'MyProject.egg-info/SOURCES.txt'
writing manifest file 'MyProject.egg-info/SOURCES.txt'
running build_ext
Creating /Users/rick446/src/arborian-classes/data/MyProject/env/lib/python3.7/site-packages/MyProject.egg-link (link to .)
MyProject 0.1 is already the active version in easy-install.pth
Installing my-greet script to /Users/rick446/src/arborian-classes/data/MyProject/env/bin

Installed /Users/rick446/src/arborian-classes/data/MyProject
Processing dependencies for MyProject==0.1
Searching for numpy==1.16.6
Best match: numpy 1.16.6
Processing numpy-1.16.6-py3.7-macosx-10.14-x86_64.egg
numpy 1.16.6 is already the active version in easy-install.pth
Installi

In [27]:
!data/MyProject/env/bin/my-greet

This is the __init__ file for mypackage
This is an awesome template that greets you.

Hello, unknown human!



In [28]:
!data/MyProject/env/bin/my-greet class

This is the __init__ file for mypackage
This is an awesome template that greets you.

Hello, class!



In [29]:
cat data/MyProject/env/bin/my-greet

#!/Users/rick446/src/arborian-classes/data/MyProject/env/bin/python
# EASY-INSTALL-ENTRY-SCRIPT: 'MyProject','console_scripts','my-greet'
__requires__ = 'MyProject'
import re
import sys
from pkg_resources import load_entry_point

if __name__ == '__main__':
    sys.argv[0] = re.sub(r'(-script\.pyw?|\.exe)?$', '', sys.argv[0])
    sys.exit(
        load_entry_point('MyProject', 'console_scripts', 'my-greet')()
    )


# Registering with PyPI

You'll need to create an account at http://pypi.org

In [38]:
%%file data/MyProject/setup.cfg
[metadata]
;; change name to make it unique
name = ProductionalizingProject-1
url = https://github.com/DevelopIntelligence
author = Some Person
author_email = somebody@example.com
version = 0.5.3
description = This should be a short description of our project
long_description = file: README.md
classifiers =
    Programming Language :: Python :: 3
    Programming Language :: Python :: 3.7
keywords = test, class

[options]
packages = mypackage
install_requires = 
    jupyter
    flask
    numpy>=1.16.0<1.17
    
[options.package_data]
* = *.txt

Overwriting data/MyProject/setup.cfg


In [39]:
%%bash
cd data/MyProject
rm dist/*   # clean up old distributions
source env/bin/activate
pip install twine
python setup.py sdist
twine upload dist/*

Looking in links: /Users/rick446/src/wheelhouse
  Using cached https://files.pythonhosted.org/packages/99/94/08b3b933c611416dad89c8abcc94a6d6c29e8609987235b6e7f10b42de82/twine-3.1.1-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/47/55/fd9170ba08a1a64a18a7f8a18f088037316f2a41be04d2fe6ece5a653e8f/tqdm-4.43.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/60/ef/7681134338fc097acef8d9b2f8abe0458e4d87559c689a8c306d0957ece5/requests_toolbelt-0.9.1-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/8b/03/a00d504808808912751e64ccf414be53c29cad620e3de2421135fcae3025/importlib_metadata-1.5.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/e6/d5/451b913307b478c49eb29084916639dc53a88489b993530fed0a66bab8b9/pkginfo-1.5.0.1-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/b2/34/bfcb43cc0ba81f527bc4f40ef41ba2ff4080e047acb0586b56b3d017ace4/zipp-3.1.0-py3-none-any.whl
  

You should consider upgrading via the 'pip install --upgrade pip' command.


In [45]:
!data/MyProject/env/bin/twine --help


usage: twine [-h] [--version] {check,register,upload}

positional arguments:
  {check,register,upload}

optional arguments:
  -h, --help            show this help message and exit
  --version             show program's version number and exit


In [46]:
!python -m venv env-tmp

In [47]:
%%bash
source env-tmp/bin/activate
pip install ProductionalizingProject-1

Looking in links: /Users/rick446/src/wheelhouse
  Using cached https://files.pythonhosted.org/packages/81/14/6d7c914dac1cb2b596d2adace4aa4574d20c0789780f1339d535e69e271f/numpy-1.18.2-cp37-cp37m-macosx_10_9_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/d6/de/2a0bda85367881e27370a206a561326a99fbb05ab9402f4c4ad59761eec4/qtconsole-4.7.1-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/78/13/cc35fa7aef711d1b384ed8cf4304d1b16c38d8f91c1e8fb1466eae96f828/ipykernel-5.2.0-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/63/0d/df2d17cdf389cea83e2efa9a4d32f7d527ba78667e0153a8e676e957b2f7/jupyter_core-4.6.3-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/95/84/119a46d494f008969bf0c775cb2c6b3579d3c4cc1bb1b41a022aa93ee242/tornado-6.0.4.tar.gz
  Using cached https://files.pythonhosted.org/packages/2a/55/b6ae78c63ab164715aa4915caed44b7bdfb03af0a421edbbf198b6f9371d/pyzmq-19.0.0-cp37-cp37m-macosx_10_9_x86_64

You should consider upgrading via the 'pip install --upgrade pip' command.


In [48]:
%%bash
source env-tmp/bin/activate
pip freeze

-f /Users/rick446/src/wheelhouse
appnope==0.1.0
attrs==19.3.0
backcall==0.1.0
bleach==3.1.3
click==7.1.1
decorator==4.4.2
defusedxml==0.6.0
entrypoints==0.3
Flask==1.1.1
importlib-metadata==1.5.0
ipykernel==5.2.0
ipython==7.13.0
ipython-genutils==0.2.0
ipywidgets==7.5.1
itsdangerous==1.1.0
jedi==0.16.0
Jinja2==2.11.1
jsonschema==3.2.0
jupyter==1.0.0
jupyter-client==6.1.0
jupyter-console==6.1.0
jupyter-core==4.6.3
MarkupSafe==1.1.1
mistune==0.8.4
nbconvert==5.6.1
nbformat==5.0.4
notebook==6.0.3
numpy==1.18.2
pandocfilters==1.4.2
parso==0.6.2
pexpect==4.8.0
pickleshare==0.7.5
ProductionalizingProject-1==0.5.3
prometheus-client==0.7.1
prompt-toolkit==3.0.4
ptyprocess==0.6.0
Pygments==2.6.1
pyrsistent==0.15.7
python-dateutil==2.8.1
pyzmq==19.0.0
qtconsole==4.7.1
QtPy==1.9.0
Send2Trash==1.5.0
six==1.14.0
terminado==0.8.3
testpath==0.4.4
tornado==6.0.4
traitlets==4.3.3
wcwidth==0.1.9
webencodings==0.5.1
Werkzeug==1.0.0
widgetsnbextension==3.5.1
zipp==3.1.0


In [49]:
!rm -r env-tmp

# Lab

Open [packaging lab][packaging-lab]

[packaging-lab]: ./packaging-lab.ipynb